In [10]:
from importlib import reload
from environment import lmmsabr
from environment.lmmsabr import make_nss_yield_df, compute_6m_forward_dataframe
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=10, suppress=True)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
reload(lmmsabr)




hed_expiry = 2
liab_expiry = 1

def generate_datasets(hed_expiry, liab_expiry): 



    hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
    liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
    lmm = lmmsabr.LMMSABR(imm=True,tenor=5, resolution=126, tau=0.5,sim_time=0.25,  swap_hedge_expiry=hed_expiry,swap_client_expiry=liab_expiry, beta=0.5)

    # Rebonato configuration
    rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
    theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
    phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}
    
    # fixed 5pct curve
    df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
    df[:] = 0.05
    # set up starting conditions
    lmm.sample_starting_conditions(df,
                                rho_kwargs=rho_config,
                                theta_kwargs=theta_config,
                                phi_kwargs=phi_config)
    lmm.prime()
    
    suffix = f"_flat_5pct_curve_{hed_expiry_str}_{liab_expiry_str}"
    lmm.generate_episodes_imm(n_episodes=50_000, min_stress_prob=0.0, max_stress_prob=0.0, out_dir=f'data/calm{suffix}')
    lmm.generate_episodes_imm(n_episodes=50_000, min_stress_prob=1.0, max_stress_prob=1.0, out_dir=f'data/stress{suffix}')
    lmm.generate_episodes_imm(n_episodes=45_000, min_stress_prob=0.0, max_stress_prob=0.8, out_dir=f'data/curriculum{suffix}') # only 20000, eval will be 50/50
    #lmm.generate_episodes_imm(n_episodes=5000,   min_stress_prob=0.5, max_stress_prob=0.5, out_dir=f'data/5050{suffix}')

the correlation will stay constant across regimes

In [ ]:
#generate_datasets(2,1)
#generate_datasets(0.5,1)

In [ ]:
def make_plots(lmm):
    # lmm.simulate(seed=0)
    # lmm.get_swap_matrix()
    # lmm.get_sabr_params_imm()
    # lmm.plot(lmm.h_mat)
    # lmm.plot(lmm.g_mat)
    # print("f_sim")
    # lmm.plot(lmm.f_sim[:,lmm.ids_fwd_canon])
    # print("s_mat")
    # lmm.plot(lmm.k_mat[:,lmm.ids_fwd_canon]*lmm.g_mat[:,lmm.ids_fwd_canon])
    # plt.show()
    # plt.plot(lmm.V)
    # plt.show()
    # plt.plot(lmm.alpha)
    # plt.show()
    # plt.plot(lmm.phi)
    # plt.show()
    # n_sim = 100
    # swap_paths_hed = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    # swap_paths_liab  = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    # for i in range(n_sim):
    #     lmm.simulate()
    #     lmm.get_swap_matrix()
    #     swap_paths_hed[i] = lmm.swap_sim[:,0]
    #     swap_paths_liab[i] = lmm.swap_sim[:,1]
    # plt.plot(swap_paths_hed.T)
    # plt.show()
    # plt.plot(swap_paths_liab.T)
    # plt.show()
    struct = lmm.sabr_structured_data()
    #lt.plot(struct.hedge.swap.rate)
    print(np.std(np.diag(struct.hedge.swaption.vega)))
    plt.plot(np.diag(struct.hedge.swaption.vega))
    plt.show()

### Calm regime

Start by plotting the interesting features in the calm regime

In [28]:
reload(lmmsabr)
hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
lmm = lmmsabr.LMMSABR(imm=True,tenor=5, resolution=126, tau=0.5,sim_time=0.25,  swap_hedge_expiry=0.5,swap_client_expiry=1, beta=0.5)

# Rebonato configuration
rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}

# fixed 5pct curve
df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
df[:] = 0.05
# set up starting conditions
lmm.sample_starting_conditions(df,
                            rho_kwargs=rho_config,
                            theta_kwargs=theta_config,
                            phi_kwargs=phi_config)
lmm.prime()
0
pnl_list = []
pnl_list_liab = []
#for i in range(20000):
lmm.simulate()
lmm.get_swap_matrix()
#pnl_list.append(lmm.swap_sim[:,0])
#pnl_list_liab.append()
#print(np.sum(np.array(pnl_list)),np.sum(np.array(pnl_list_liab)))


t_max set to 6.25
n_curves is none, setting n_curves to n_samples=1
random_curves set to False, using the first n_curves of the dataframe
creating df init for n_curves=1
done creating df_init
sampling
(0.5727, 0.0002, 2.3035, 0.2757)
(1.1138, 0.0002, 1.9833, 0.3069)
Creating combined covariance matrix...


(array([[0.05        , 0.05        ],
        [0.0491121592, 0.0490789965],
        [0.0492013415, 0.0491739818],
        [0.0482913648, 0.0482859044],
        [0.0470087351, 0.0469734303],
        [0.0463328738, 0.0462604814],
        [0.0451235423, 0.0450279613],
        [0.0450092788, 0.0449143785],
        [0.0452499765, 0.0451576392],
        [0.0448666074, 0.0447636605],
        [0.0443836708, 0.0442873783],
        [0.0448018311, 0.0447389227],
        [0.0444474024, 0.0443902511],
        [0.0438170659, 0.0437577808],
        [0.0440347151, 0.0439980386],
        [0.0437590196, 0.0437193777],
        [0.0438601766, 0.0438645688],
        [0.0441004955, 0.044129478 ],
        [0.0442021303, 0.0442600926],
        [0.0454405731, 0.045527486 ],
        [0.0448801931, 0.044928675 ],
        [0.0443377955, 0.0444115668],
        [0.0450038207, 0.0451022742],
        [0.0446100549, 0.0446988193],
        [0.0445359533, 0.0446204832],
        [0.0449526668, 0.0450471675],
        [0.0

In [29]:

print(lmm.P[0,lmm.ids_fwd_canon[1]]-lmm.P[0,lmm.ids_fwd_canon[2]])
print(lmm.tau*lmm.f_sim[0,lmm.ids_fwd_canon[2]]*lmm.P[0,lmm.ids_fwd_canon[2]])

0.023214985272993593
0.023214985272993725


In [109]:

hed_expiry_str = f"{int(12*hed_expiry)}m" if type(hed_expiry) == float else f"{hed_expiry}y"
liab_expiry_str = f"{int(12*liab_expiry)}m" if type(liab_expiry) == float else f"{liab_expiry}y"
lmm = lmmsabr.LMMSABR(imm=True,tenor=1, resolution=126, tau=0.5,sim_time=0.5,  swap_hedge_expiry=0.5,swap_client_expiry=1, beta=0.5)

# Rebonato configuration
rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}

# fixed 5pct curve
df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
df[:] = 0.05
# set up starting conditions
lmm.sample_starting_conditions(df,
                            rho_kwargs=rho_config,
                            theta_kwargs=theta_config,
                            phi_kwargs=phi_config)
lmm.prime()
lmm.simulate()
lmm.get_swap_matrix()
lmm.sabr_structured_data().hedge.swaption.price

t_max set to 2.5
n_curves is none, setting n_curves to n_samples=1
random_curves set to False, using the first n_curves of the dataframe
creating df init for n_curves=1
done creating df_init
sampling
(0.5727, 0.0002, 2.3035, 0.2757)
(1.1138, 0.0002, 1.9833, 0.3069)
Creating combined covariance matrix...


array([[0.0015977885,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan

In [ ]:

idx_swapt = np.arange(lmm.swap_sim_shape[0])
swaption_val_0Tn = lmm.sabr_structured_data().hedge.swaption.price[idx_swapt,idx_swapt]
P0T = lmm.P[0,-1]
n_sim = 10000
mc_mat = np.zeros((n_sim, len(swaption_val_0Tn)))
for i in range(n_sim):
    lmm.simulate()
    lmm.get_swap_matrix()
    PTiTn = lmm.P[-1, -1] # last row is expiry, last column is terminal forward
    swap_value = lmm.annuity[:,0] *(lmm.swap_sim[:,0] - lmm.swap_sim[0,0] )
    swap_val_0Tn =   swap_value[-1] - swap_value
    H = np.maximum(swap_val_0Tn,0)
    MC_price = P0T* H / PTiTn
    mc_mat[i] = MC_price




In [150]:




swaption_val_0Tn -np.mean(mc_mat,axis=0)
np.std(mc_mat,axis=0)

array([0.0022135953, 0.0022053054, 0.0021967463, 0.0021886389, 0.0021792149, 0.002167807 , 0.0021539807, 0.0021446082, 0.0021284714, 0.0021128935, 0.0021066713, 0.0020994968, 0.0020853265, 0.002070899 , 0.0020560875, 0.0020391819, 0.0020302763, 0.0020115177, 0.0020004642, 0.0019852004, 0.0019753104, 0.001963726 , 0.0019564342, 0.0019389879, 0.0019356456, 0.0019262094, 0.0019150721, 0.0019025515, 0.0018859514, 0.001874713 , 0.0018626586, 0.0018531755, 0.0018419136, 0.0018280088, 0.0018175807, 0.0018097078, 0.0018017041, 0.0017833211, 0.0017662454, 0.0017560446, 0.0017391834, 0.0017299334, 0.0017173977, 0.0017054942, 0.0016894065, 0.0016856221, 0.0016811738, 0.0016698748, 0.0016569816, 0.001644569 , 0.0016344829, 0.001625972 , 0.001612081 , 0.00159249  , 0.0015862604, 0.0015793336, 0.0015703285, 0.0015570644, 0.0015466843, 0.0015386226, 0.0015227405, 0.001505927 , 0.0014998567, 0.0014801343, 0.0014659169, 0.0014489897, 0.001432021 , 0.0014212678, 0.0014077288, 0.0013934801, 0.0013845277,

array([ 0.          ,  0.0003168557,  0.0000200175, -0.0002310108, -0.0009315433, -0.0007703429, -0.0014893611, -0.0013336554, -0.0014936133, -0.0015863571, -0.001942602 , -0.002252575 , -0.0017318215, -0.0019673165, -0.0019497003, -0.0010599254, -0.0015854892, -0.0015838808, -0.001633581 , -0.0018775122, -0.0029526855, -0.0034365281, -0.0038649742, -0.0043416609, -0.0042421792, -0.0050920931, -0.0054510346, -0.0054327221, -0.0054616607, -0.0053480625, -0.0047138235, -0.0049352208, -0.003775107 , -0.00431756  , -0.0054161267, -0.0056646786, -0.006363399 , -0.0059406682, -0.0053869232, -0.0054364213, -0.0057021191, -0.005513792 , -0.0060945478, -0.0059317901, -0.0055809985, -0.005288699 , -0.0057704307, -0.0052560769, -0.0051079686, -0.0054449546, -0.0053836808, -0.0054010594, -0.0058627201, -0.006803738 , -0.0070754353, -0.0069745416, -0.0067140762, -0.0056976905, -0.0062834595, -0.0061794767, -0.006297229 , -0.0060486361, -0.0058210767, -0.006016146 , -0.0052892124, -0.0051695304, -0.

In [115]:
lmm.sabr_structured_data().hedge.swaption.price[1,1]

0.0016254997778157815

In [108]:
price_analytic- P0T*np.mean(np.array(mc_list))

-1.1978120814585061e-05

In [ ]:
terminal_forward = 0
for i in range(100):
    lmm.simulate()
    terminal_forward += np.sum(lmm.f_sim[:,-1])


In [16]:
f_sim_new = lmm.f_sim.copy()
f_sim_new[:,0] = f_sim_new=0.0
lmm.f_sim

array([[0.05        ,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan,          nan

### Stressed regime

In [ ]:

# lmm.switch_regime(stressed=False)
# lmm.simulate()
# lmm.get_swap_matrix()
# make_plots(lmm)

### Curriculum calm->stress regime

### 50/50 test set for evaluation across training methods